In [1]:
import torch
print(torch.cuda.is_available())

True


c:\Users\Hi\anaconda3\envs\KLTN\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

# INPUT TEXT MUST BE ALREADY WORD-SEGMENTED!
sentence = 'Chúng_tôi là những nghiên_cứu_viên .'  

input_ids = torch.tensor([tokenizer.encode(sentence)])

with torch.no_grad():
    features = phobert(input_ids)  # Models outputs are now tuples
features

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 3.8361e-02,  7.0703e-01, -1.3202e-01,  ..., -9.7446e-02,
           2.5193e-01,  3.4828e-01],
         [ 2.1041e-01,  2.3984e-01,  9.1059e-03,  ..., -3.2353e-04,
          -1.7492e-01,  4.0126e-02],
         [ 2.3745e-01,  9.8414e-03, -1.6509e-01,  ..., -4.3378e-02,
          -7.5783e-02,  4.6837e-02],
         ...,
         [ 2.3041e-01,  3.7583e-01,  1.7602e-02,  ...,  7.8472e-02,
           1.8661e-01,  5.2053e-02],
         [-2.3020e-01,  5.0276e-01,  1.0913e-01,  ..., -7.3260e-02,
           1.4339e-01,  1.8320e-01],
         [ 1.8813e-01,  6.2870e-01, -2.4809e-01,  ..., -4.8115e-02,
           1.6404e-01,  4.7204e-01]]]), pooler_output=tensor([[ 5.9168e-02, -1.4942e-01,  3.2264e-02,  1.1025e-01, -9.1165e-02,
          3.8922e-02, -1.7292e-01, -3.2285e-02,  1.3863e-01, -2.0987e-01,
          3.2109e-01, -3.6521e-02,  1.0351e-01,  1.2799e-01,  4.0665e-02,
          9.4652e-02, -7.3467e-02, -1.8222e-01, -1.798

# setup input process and tokenizer

## text preprocessing

In [10]:
import re
from underthesea import word_tokenize, text_normalize

# text cleaning
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_html_tags(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

def remove_non_alpha(text):
    pattern = re.compile(r'[^a-zA-Z0-9áàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđĐ ]')
    return pattern.sub(r'', text)

# text nomalization
def to_lowercase(text):
    return text.lower()

def standardize_spaces(text):
    return re.sub(r'\s+', ' ', text).strip()

def preprocess_text(text):
    text = remove_urls(text)
    text = remove_html_tags(text)
    text = remove_non_alpha(text)
    text = to_lowercase(text)
    text = standardize_spaces(text)
    text = text_normalize(text)
    return text

In [14]:
text = "Chúng tôi là những nghiên cứu viên. Check out: https://example.com <br> Đây là một ví dụ!"
cleaned_text = preprocess_text(text)
tokens = word_tokenize(cleaned_text, format="text")
print(tokens)


chúng_tôi là những nghiên_cứu_viên check out đây là một ví_dụ


In [31]:
import pandas as pd 


path_dataset = "dataset.csv"
# Read the CSV file
df = pd.read_csv(path_dataset)

# Convert all entries in the 'Text' column to strings
df['Text'] = df['Text'].astype(str)


# Preprocess the 'text' column
df['processed_text'] = df['Text'].apply(preprocess_text)

# Tokenize the preprocessed text
df['word_tokens'] = df['processed_text'].apply(lambda x: word_tokenize(x, format="text"))


In [32]:
# Display the processed data
df['word_tokens']

0       theo sankei sports sáng nay thủ_tướng nhật_bản...
1       nhìn bức tranh bạn có_thể thấy vòng_tròn đang ...
2       sau khi vụ án xảy ra tất_cả mọi người đều khôn...
3       xét_nghiệm ban_đầu từ các trẻ nhập_viện cho th...
4       sáng ngày 289 theo nguồn tin riêng từ ubnd thà...
                              ...                        
1901    mẫu xe hạng a của thương_hiệu hàn_quốc nhận đư...
1902    theo hđxx bị_cáo ngô_minh khâm_giữ vai_trò chủ...
1903    cổ động_viên real_madrid bất_mãn khi hậu_vệ da...
1904    một bác_sĩ trong êkíp đỡ_sinh phải bế và tỳ_cả...
1905    thay_vì nỗ_lực kiếm tiền nhiều người trẻ mới t...
Name: word_tokens, Length: 1906, dtype: object

## tokenize

In [147]:
from transformers import AutoConfig

# Load the configuration of the model
config = AutoConfig.from_pretrained("vinai/phobert-base-v2")
# Specify the number of labels for sequence classification
config.num_labels = 2
config.batch_size = 8 

# Print the maximum input length
print(config)


RobertaConfig {
  "_name_or_path": "vinai/phobert-base-v2",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "batch_size": 8,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 258,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "PhobertTokenizer",
  "torch_dtype": "float32",
  "transformers_version": "4.33.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 64001
}



In [148]:
from transformers import AutoTokenizer
from underthesea import word_tokenize, text_normalize


tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")
sentences = list(df['word_tokens'])


labels = list(df['Label'])
inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True, max_length=config.max_position_embeddings)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [151]:
print(len(inputs))

3


# make data loader

In [152]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

# Split the data into training and validation sets while maintaining the label distribution
train_input_ids, val_input_ids, train_attention_mask, val_attention_mask, train_labels, val_labels = train_test_split(
    inputs.input_ids, inputs.attention_mask, torch.tensor(labels), test_size=0.2, random_state=42, stratify=torch.tensor(labels)
)

# Create TensorDatasets for training and validation sets
train_dataset = TensorDataset(train_input_ids, train_attention_mask, train_labels)
val_dataset = TensorDataset(val_input_ids, val_attention_mask, val_labels)

# Create data loaders for training and validation sets
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [154]:
# For the training dataloader
train_samples = len(train_dataloader.dataset)

# For the validation dataloader
val_samples = len(val_dataloader.dataset)

print(f"Number of batches in the training set: {len(train_dataloader)}")
print(f"Number of batches in the training set: {len(val_dataloader)}")

print(f"Number of samples in the training set: {train_samples}")
print(f"Number of samples in the validation set: {val_samples}")


Number of batches in the training set: 96
Number of batches in the training set: 24
Number of samples in the training set: 1524
Number of samples in the validation set: 382


# for customer fine tunning layer

In [ ]:
import torch.nn as nn

class CustomClassificationHead(nn.Module):
    def __init__(self, hidden_size, num_labels):
        super().__init__()
        self.dense = nn.Linear(hidden_size, hidden_size)
        self.dropout = nn.Dropout(0.1)
        self.out_proj = nn.Linear(hidden_size, num_labels)

    def forward(self, features, **kwargs):
        x = self.dropout(features)
        x = self.dense(x)
        x = nn.ReLU()(x)
        x = self.dropout(x)
        logits = self.out_proj(x)
        return logits

# Replace the classification head in PhoBERT
model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base-v2", num_labels=2)
model.classifier = CustomClassificationHead(model.config.hidden_size, 2)


# use default setup for fine tunning layer

In [157]:

for batch in train_dataloader:
    input_ids, attention_mask, labels = batch
    print(input_ids.shape, attention_mask.shape, labels.shape)
    break

torch.Size([16, 258]) torch.Size([16, 258]) torch.Size([16])


In [159]:
import torch.optim as optim
from transformers import AutoConfig, AutoModelForSequenceClassification

# Load the model using the modified configuration
model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base-v2", config=config)

model.train()
for batch in train_dataloader:
    input_ids, attention_mask, labels = batch
    print(input_ids.shape, attention_mask.shape, labels.shape)
    print(model(input_ids, attention_mask=attention_mask, labels=labels))
    break


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


torch.Size([16, 258]) torch.Size([16, 258]) torch.Size([16])


IndexError: index out of range in self

In [109]:
import torch.optim as optim
from transformers import AutoConfig, AutoModelForSequenceClassification

# Load the model using the modified configuration
model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base-v2", config=config)

# Optimizer
optimizer = optim.AdamW(model.parameters(), lr=1e-5)
num_epochs = 4

# Assuming you have a validation dataloader named val_dataloader
for epoch in range(num_epochs):
    model.train()
    # Training
    total_loss = 0
    for batch in train_dataloader:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    
    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs} - Training Loss: {avg_train_loss:.4f}")
    
    # Validation
    model.eval()
    val_loss = 0
    correct_predictions = 0
    with torch.no_grad():
        model.val()
        for batch in val_dataloader:
            input_ids, attention_mask, labels = batch
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()
            
            logits = outputs.logits
            _, preds = torch.max(logits, dim=1)
            correct_predictions += torch.sum(preds == labels)
    
    avg_val_loss = val_loss / len(val_dataloader)
    val_accuracy = correct_predictions.double() / len(val_dataloader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs} - Validation Loss: {avg_val_loss:.4f} - Validation Accuracy: {val_accuracy:.4f}")
    
    model.train()



Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


IndexError: index out of range in self

# demo test

In [ ]:
model.eval()
with torch.no_grad():
    new_sentence = "This is a new article to check."
    inputs = tokenizer(new_sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
    logits = model(**inputs).logits
    prediction = torch.argmax(logits, dim=1).item()
    print("Fake" if prediction == 1 else "Real")
